In [26]:
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))

Python: 2.7.13 |Anaconda 4.3.1 (64-bit)| (default, Dec 20 2016, 23:09:15) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
scipy: 0.18.1
numpy: 1.11.3
matplotlib: 2.0.0
pandas: 0.19.2
sklearn: 0.18.1


In [27]:
# Load libraries
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVC

In [28]:
import datetime
from datetime import date,time
import time

In [29]:
pd.options.mode.chained_assignment = None  # default='warn'

In [30]:
def time_num(date):
    intervalo = 30
    
    hora = date.hour
    minuto = date.minute
    
    num_minutes = hora*60 + minuto
    
    time_bin = num_minutes / intervalo
    hour_bin = num_minutes / 60
    min_bin = (time_bin * intervalo) % 60
    
    return ((hour_bin * 60 + min_bin + intervalo) / 2.0)/(60*24)

In [55]:
dataset = pd.read_csv('data/trip_train.csv')
weather = pd.read_csv('data/weather.csv')
station = pd.read_csv('data/station.csv')
test = pd.read_csv('data/trip_test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119998 entries, 0 to 119997
Data columns (total 10 columns):
id                    119998 non-null int64
start_date            119998 non-null object
start_station_name    119998 non-null object
start_station_id      119998 non-null int64
end_date              119998 non-null object
end_station_name      119998 non-null object
end_station_id        119998 non-null int64
bike_id               119998 non-null int64
subscription_type     119998 non-null object
zip_code              118772 non-null object
dtypes: int64(4), object(6)
memory usage: 9.2+ MB


In [32]:
station = station.loc[:,['id','lat','long']]

In [33]:
weather.date = pd.to_datetime(weather.date)

In [34]:
weather = weather.loc[:,['date','mean_temperature_f']].groupby('date').mean()
weather = pd.DataFrame(weather.to_records())

In [35]:
dataset = dataset.loc[(dataset.duration < 86400) & (dataset.duration > 60),:]

In [36]:
prueba = dataset.head(15000)
prueba.describe()

,id,duration,start_station_id,end_station_id,bike_id
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,463000.576200,977.325800,58.095067,57.980400,427.726200
std,263317.734136,3167.134849,16.899018,17.025608,153.671141
min,4086.000000,63.000000,2.000000,2.000000,9.000000
25%,236500.500000,337.000000,50.000000,50.000000,334.000000
50%,459174.500000,508.000000,62.000000,62.000000,441.000000
75%,694277.000000,751.000000,70.000000,70.000000,546.000000
max,913416.000000,84070.000000,84.000000,84.000000,878.000000


In [37]:
prueba.start_date = pd.to_datetime(prueba.start_date)
#prueba.end_date = pd.to_datetime(prueba.end_date)

In [38]:
prueba['time_num'] = prueba.start_date.apply(lambda x: time_num(x))

In [39]:
prueba['date'] = prueba.start_date.apply(lambda x: datetime.datetime(x.year, x.month, x.day,0,0))

In [40]:
prueba = pd.merge(prueba,weather,on='date',how='inner')

In [41]:
prueba

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,time_num,date,mean_temperature_f
0,907649,396,2015-08-27 08:36:00,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,0.187500,2015-08-27,75.2
1,908115,486,2015-08-27 12:24:00,Embarcadero at Bryant,54,8/27/2015 12:32,Embarcadero at Vallejo,48,493,Subscriber,94105,0.260417,2015-08-27,75.2
2,907549,1027,2015-08-27 08:10:00,San Francisco Caltrain 2 (330 Townsend),69,8/27/2015 8:27,Market at Sansome,77,373,Subscriber,94105,0.177083,2015-08-27,75.2
3,907722,354,2015-08-27 08:53:00,Market at Sansome,77,8/27/2015 8:59,2nd at Folsom,62,395,Subscriber,94612,0.187500,2015-08-27,75.2
4,908290,209,2015-08-27 15:22:00,San Francisco Caltrain 2 (330 Townsend),69,8/27/2015 15:26,Townsend at 7th,65,313,Subscriber,94044,0.322917,2015-08-27,75.2
5,907615,849,2015-08-27 08:25:00,San Francisco Caltrain 2 (330 Townsend),69,8/27/2015 8:39,Clay at Battery,41,418,Subscriber,94303,0.177083,2015-08-27,75.2
6,907916,981,2015-08-27 09:49:00,Harry Bridges Plaza (Ferry Building),50,8/27/2015 10:05,Broadway St at Battery St,82,636,Subscriber,94114,0.208333,2015-08-27,75.2
7,908212,239,2015-08-27 13:58:00,Temporary Transbay Terminal (Howard at Beale),55,8/27/2015 14:02,Clay at Battery,41,529,Subscriber,94965,0.291667,2015-08-27,75.2
8,909018,698,2015-08-27 20:02:00,Embarcadero at Folsom,51,8/27/2015 20:14,San Francisco Caltrain (Townsend at 4th),70,576,Customer,85601,0.427083,2015-08-27,75.2
9,908705,414,2015-08-27 17:41:00,Market at 4th,76,8/27/2015 17:48,San Francisco Caltrain (Townsend at 4th),70,549,Subscriber,94303,0.375000,2015-08-27,75.2


In [42]:
prueba = pd.merge(prueba,station,left_on='start_station_id',right_on='id',how='inner')

In [43]:
prueba.describe()

,id_x,duration,start_station_id,end_station_id,bike_id,time_num,mean_temperature_f,id_y,lat,long
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,463000.576200,977.325800,58.095067,57.980400,427.726200,0.288135,62.277653,58.095067,37.746324,-122.360588
std,263317.734136,3167.134849,16.899018,17.025608,153.671141,0.094885,6.771272,16.899018,0.124254,0.125411
min,4086.000000,63.000000,2.000000,2.000000,9.000000,0.010417,39.200000,2.000000,37.329732,-122.418954
25%,236500.500000,337.000000,50.000000,50.000000,334.000000,0.197917,57.600000,50.000000,37.776617,-122.403234
50%,459174.500000,508.000000,62.000000,62.000000,441.000000,0.291667,62.200000,62.000000,37.785299,-122.396236
75%,694277.000000,751.000000,70.000000,70.000000,546.000000,0.364583,67.800000,70.000000,37.792251,-122.394203
max,913416.000000,84070.000000,84.000000,84.000000,878.000000,0.500000,80.400000,84.000000,37.804770,-121.877349


In [44]:
prueba['hour_start'] = prueba.start_date.apply(lambda x: x.hour)
#prueba['hour_end'] = prueba.end_date.apply(lambda x: x.hour)
prueba['weekend'] = prueba.start_date.apply(lambda x: 1 if x.weekday() >= 5 else 0)

In [ ]:
prueba

In [45]:
prueba = prueba.loc[:,['duration','mean_temperature_f','time_num','weekend','lat','long']]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549678 entries, 0 to 549960
Data columns (total 11 columns):
id                    549678 non-null int64
duration              549678 non-null int64
start_date            549678 non-null object
start_station_name    549678 non-null object
start_station_id      549678 non-null int64
end_date              549678 non-null object
end_station_name      549678 non-null object
end_station_id        549678 non-null int64
bike_id               549678 non-null int64
subscription_type     549678 non-null object
zip_code              544300 non-null object
dtypes: int64(5), object(6)
memory usage: 50.3+ MB


In [ ]:
prueba

In [ ]:
#start_station_name = pd.get_dummies(prueba['start_station_name'])
#end_station_name = pd.get_dummies(prueba['end_station_name'])

In [ ]:
#print start_station_name.shape
#print end_station_name.shape

In [ ]:
#prueba = prueba.drop('start_station_name',axis=1)
#prueba = prueba.drop('end_station_name',axis=1)
#prueba.shape

In [ ]:
#prueba = prueba.join(start_station_name)

In [ ]:
#columnas = [x +  '_end' for x in list(end_station_name.columns.tolist())]
#end_station_name.columns = columnas

In [ ]:
#prueba = prueba.join(end_station_name)

In [ ]:
#days = pd.get_dummies(prueba['weekday'])
#prueba = prueba.join(days)
#prueba = prueba.drop('weekday',axis=1)
#prueba

In [46]:
columnas = prueba.columns.tolist()
columnas = columnas[1:len(columnas)]
columnas.append('duration')
prueba = prueba[columnas]
len(prueba.columns.tolist())

6

In [47]:
from sklearn import preprocessing

In [48]:
array = prueba.values
X = array[:,0:len(prueba.columns.tolist())-1]
Y = array[:,len(prueba.columns.tolist())-1]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [49]:
seed = 7
scoring = 'accuracy'

In [ ]:
#models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
#results = []
#names = []
#for name, model in models:
#    kfold = model_selection.KFold(n_splits=10, random_state=seed)
#    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
#    results.append(cv_results)
#   names.append(name)
#    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
#    print(msg)

In [50]:
start_time = time.time()
models = []
#models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
for name, model in models:
    model.fit(X_train,Y_train)
    predictions = model.predict(X_validation)
    msg = "%s -> Mean Squared Error: %f" % (name, mean_squared_error(Y_validation, predictions))
    print(msg)
print("--- %s seconds ---" % (time.time() - start_time))

LDA -> Mean Squared Error: 36990927.027333
KNN -> Mean Squared Error: 8119117.412333
CART -> Mean Squared Error: 20433628.956333
NB -> Mean Squared Error: 8463239.426333
--- 4.81606292725 seconds ---


In [54]:
start_time = time.time()
lda = RandomForestRegressor()
#lda.fit(X_train, Y_train)
#predictions = lda.predict(X_validation)
parameters = {
    'n_estimators':[50],
    'max_features':['auto'],
    'max_depth':[50]}
best = cv_optimize
print Y_validation[0:15]
print predictions[0:15]
#print accuracy_score(Y_validation,predictions)
print(mean_squared_error(Y_validation, predictions))
print("--- %s seconds ---" % (time.time() - start_time))

[ 1079.   229.   324.   355.  1366.   285.   580.  7155.   620.   203.
   677.  3333.   515.   372.   519.]
[  1259.8           586.            498.2         10245.2          2713.1
    286.55          747.4         13965.6           619.6           429.7
    602.           1152.1           601.23333333    412.06666667
    673.46666667]
10236490.8663
--- 0.603191137314 seconds ---


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_train_scaled = preprocessing.scale(X_train)
X_validation_scaled = preprocessing.scale(X_validation)
X_train_normalized = preprocessing.normalize(X_train)

In [ ]:
lda.fit(X_train_scaled,Y_train)
predictions_scaled = lda.predict(X_validation_scaled)
print mean_squared_error(Y_validation,predictions_scaled)

In [ ]:
lda.fit(X_train_normalized,Y_train)
prediction_normalized = lda.predict(X_validation)
print prediction_normalized
print Y_validation
print mean_squared_error(Y_validation,prediction_normalized)